In [ ]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

#### Adapter Layer

In [ ]:
def csv_to_df(bucket, key, decoding='utf-8',sep=','):
    csv_obj = bucket.Object(key=key).get("Body").read().decode(decoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

def write_df_s3(bucket, df, key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.get_value(), key=key)
    return True

def list_files_and_prefix(bucket, prefix):
    files = [obj.key for obj in bucket.objects.filter(Prefix=prefix)]
    return files

#### Application Layer

In [ ]:
def extract(bucket, date_list):
    files = [key for date in date_list for key in list_files_and_prefix(bucket, date)]
    df = pd.concat([csv_to_df(bucket, obj) for obj in files], ignore_index=True)
    return df

def transform_report1(df, columns, arg_date):
    df = df.loc[:, columns]
    df.dropna(inplace=True)
    df['opening_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
    df['closing_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')
    df = df.groupby(['ISIN', 'Date'], as_index=False).agg(
        opening_price_eur=('opening_price', 'min'), 
        closing_price_eur=('closing_price', 'min'),
        minimum_price_eur=('MinPrice', 'min'),
        maximum_price_eur=('MaxPrice', 'max'),
        daily_trader_volume=('TradedVolume', 'sum')
    )
    df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    
    df['change_prev_closing_%'] = (
        df['closing_price_eur'] - df['prev_closing_price']
    ) / df['prev_closing_price'] * 100
    
    df.drop(columns=['prev_closing_price'], inplace=True)
    df = df.round(decimals=2)
    df = df[df.Date >= arg_date]
    return df

def load(bucket, df, trg_key, trg_format):
    key = trg_key + datetime.today().strftime("%Y-%m-%d_%H%M%S") + trg_format
    write_df_s3(bucket=bucket, df=df, key=key)
    return True

def etl_report1(src_bucket, trg_bucket, date_list, columns, arg_date, trg_key, trg_format):
    df = extract(src_bucket, date_list)
    df = transform_report1(df, columns, arg_date)
    load(trg_bucket, df, trg_key, trg_format)
    return True
    

In [ ]:
def return_date_list(bucket, args_date, src_format, meta_key):
    min_date = datetime.strptime(args_date, src_format).date() - timedelta(days=1)
    today = datetime.today().date()
    
    try:
    
        df_meta = csv_to_df(bucket, meta_key)
        dates = [(min_date + timedelta(timedelta=x)).strftime(src_format) for x in range(0, (today-min_date).days + 1)]
        src_dates = set(pd.to_datetime(df_meta['source_date']).dt.date)

        dates_missing = set(dates[1:]) - src_dates

        if dates_missing:
            min_date = min(set(dates[1:]) - src_dates) - timedelta(days=1)
            return_dates = [date.strftime(src_format) for date in dates if date >= min_date]
            return_min_date = min_date + timedelta(days=1).strftime(src_format)
        else:
            return_dates = []
            return_min_date = datetime(2200,1,1).date()
    
    except bucket.session.client('s3').exceptions.NoSuchKey:
        return_dates = [(min_date + timedelta(timedelta=x)).strftime(src_format) for x in range(0, (today-min_date).days + 1)]
        return_min_date = args_date
        
    return return_min_date, return_dates

#### Main Function entry point

In [1]:
def main():
    # Parameters/Configurations
    args_date = '2022-05-11'
    src_format = '%Y-%m-%d'
    src_bucket = 'xetra-1234'
    target_bucket = 'xetra-1234'
    trg_key = 'xetra_daily_report'
    trg_format = ".parquet"
    columns = ["ISIN", "Date", "Time", "StartPrice", "MaxPrice", "MinPrice", "EndPrice", "TradeVolume"]
    meta_key = 'meta_file.csv'
    
    # Init
    s3 = boto3.resource("s3")
    bucket_src = s3.Bucket(src_bucket)
    bucket_trg = s3.Bucket(target_bucket)
    
    # Run application
    extract_date, date_list = return_date_list(bucket_trg, args_date, src_format, meta_key=meta_key)
    etl_report1(bucket_src, bucket_trg, date_list, columns, extract_date, trg_key, trg_format)

In [10]:
# run application

if __name__ == "__main__":
    main()

#### Reading the uploaded file

In [ ]:
target_bucket = 'xetra-1234'
s3 = boto3.resource("s3")
bucket_trg = s3.Bucket(target_bucket)

In [ ]:
for obj in bucket_trg.objects.all():
    print(obj.key)

In [ ]:
prq_obj = bucket_trg.Object(key='xetra_daily_report_20210511_125520.parquet').get().get('Body').raed()
data = BytesIO(prq_obj)
df_report = pd.read_csv(data)

In [ ]:
df_report